**Install packages that we'll use for our ETL, including sodapy for accessing our new business data set and Google Big Query to upload our final data sets**




In [1]:
pip install --upgrade sodapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install --upgrade db-dtypes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyarrow-9.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.3 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.0
    Uninstalling pyarrow-10.0.0:
      Successfully uninstalled pyarrow-10.0.0


In [3]:
pip install --upgrade pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyarrow-10.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.4 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 10.0.0 which is incompatible.
db-dtypes 1.0.4 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 10.0.0 which is incompatible.


In [4]:
pip install --upgrade google-cloud-bigquery

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# import libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account

**We'll create a BigQuery client for use in later steps**

In [6]:
# file path to BigQuery secret
key_path = r'gcp_echow_key.json'

In [7]:
# incorporating secret setup your credentials
credentials = service_account.Credentials.from_service_account_file(key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],)
bigquery_client = bigquery.Client(credentials = credentials, project = credentials.project_id)

print(f"bigquery client name is: {bigquery_client}")
print(f"bigquery client data type is: {type(bigquery_client)}")

bigquery client name is: <google.cloud.bigquery.client.Client object at 0x7fedf332b290>
bigquery client data type is: <class 'google.cloud.bigquery.client.Client'>


**First we'll import, profile, and clean our NYC business license data from NYC Open Data**

In [8]:
# setup the host name for the API endpoint 
data_url = 'data.cityofnewyork.us'

In [9]:
# setup the data set at the API endpoint, which is the NYC business license data set
data_set = 'w7w3-xahh'

In [10]:
# loading in our NYC Open Data app token

with open(r'nyc_open_data_app_token_echow.txt') as f:
    app_token = f.read()

In [11]:
# create the client that points to the Socrata API endpoint
nyc_open_data_client = Socrata(data_url, app_token, timeout = 200)
print(f"nyc open data client name is: {nyc_open_data_client}")
print(f"nyc open data client data type is: {type(nyc_open_data_client)}")

nyc open data client name is: <sodapy.socrata.Socrata object at 0x7fedee83d590>
nyc open data client data type is: <class 'sodapy.socrata.Socrata'>


In [12]:
# Get the total number of records in our the NYC business license data set
total_record_count = nyc_open_data_client.get(data_set, select = "COUNT(*)")
print(f"total records in {data_set}: {total_record_count[0]['COUNT']}")

total records in w7w3-xahh: 277264


In [13]:
# We'll loop through target data set to pull all rows in chunks

def extract_socrata_data(chunk_size = 2500,
                         data_set = data_set,
                         where = None):
    
    # measure time this function takes
    import time
    start_time = time.time()
    
    # get total number or records
    if where == None:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     select= "COUNT(*)")[0]["COUNT"])
    else:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     where = where,
                                                     select= "COUNT(*)")[0]["COUNT"])
    
    # start at 0, empty list for results
    start = 0                   
    results = []                

    while True:

        if where == None:
            # fetch the set of records starting at 'start'
            results.extend(nyc_open_data_client.get(data_set,
                                                    offset = start,
                                                    limit = chunk_size))
            
        elif where != None:
            results.extend(nyc_open_data_client.get(data_set,
                                                    where = where,
                                                    offset = start,
                                                    limit = chunk_size))
        # update the starting record number
        start = start + chunk_size
        print("Record number for current iteration: " + str(start))

        # if we have fetched all of the records (we have reached total_records), exit loop
        if (start > total_records):
            break

    # convert the list into a pandas data frame
    data = pd.DataFrame.from_records(results)

    end_time = time.time()
    print(f"function took {round(end_time - start_time, 1)} seconds")

    print(f"the shape of your dataframe is: {data.shape}")
    return data

In [14]:
data = extract_socrata_data(chunk_size = 2500,
                            data_set = data_set)

Record number for current iteration: 2500
Record number for current iteration: 5000
Record number for current iteration: 7500
Record number for current iteration: 10000
Record number for current iteration: 12500
Record number for current iteration: 15000
Record number for current iteration: 17500
Record number for current iteration: 20000
Record number for current iteration: 22500
Record number for current iteration: 25000
Record number for current iteration: 27500
Record number for current iteration: 30000
Record number for current iteration: 32500
Record number for current iteration: 35000
Record number for current iteration: 37500
Record number for current iteration: 40000
Record number for current iteration: 42500
Record number for current iteration: 45000
Record number for current iteration: 47500
Record number for current iteration: 50000
Record number for current iteration: 52500
Record number for current iteration: 55000
Record number for current iteration: 57500
Record number 

**We will now profile our business registration data set**

In [15]:
# create and run a function to create data profiling dataframe

def create_data_profiling_df(data):
    
    # create an empty dataframe to gather information about each column
    data_profiling_df = pd.DataFrame(columns = ["column_name",
                                                "column_type",
                                                "unique_values",
                                                "duplicate_values",
                                                "null_values",
                                                "non_null_values"])

    # loop through each column to add rows to the data_profiling_df dataframe
    for column in data.columns:

        info_dict = {}

        try:
            info_dict["column_name"] = column
            info_dict["column_type"] = data[column].dtypes
            info_dict["unique_values"] = len(data[column].unique())
            info_dict["duplicate_values"] = data[column].count() - len(data[column].dropna().unique())
            info_dict["null_values"] = data[column].isna().sum()
            info_dict["non_null_values"] = data[column].count()

        except:
            print(f"unable to read column: {column}, you may want to drop this column")

        data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)

    data_profiling_df.sort_values(by = ['unique_values', "non_null_values"],
                                  ascending = [False, False],
                                  inplace=True)
    
    return data_profiling_df

In [16]:
# view data profiling dataframe
data_profiling_df = create_data_profiling_df(data = data)
data_profiling_df

unable to read column: location, you may want to drop this column


,column_name,column_type,unique_values,duplicate_values,null_values,non_null_values
0,license_nbr,object,235884,41380,0,277264
6,business_name,object,188337,88886,42,277222
13,contact_phone,object,104970,72116,100179,177085
22,longitude,object,70582,103636,103047,174217
23,latitude,object,70582,103636,103047,174217
19,bbl,object,56293,105628,115344,161920
18,bin,object,56063,105858,115344,161920
10,business_name_2,object,33513,17044,226708,50556
11,address_building,object,14210,168861,94194,183070
12,address_street_name,object,12310,171828,93127,184137


In [17]:
#check why there was an error with reading in location column 
data[['longitude', 'latitude', 'location']]

,longitude,latitude,location
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
277259,-74.1658379002205,40.545800561582986,"{'latitude': '40.545800561582986', 'longitude'..."
277260,NaN,NaN,NaN
277261,-73.87147128628813,40.80762854108828,"{'latitude': '40.80762854108828', 'longitude':..."
277262,NaN,NaN,NaN


In [18]:
# The location column seems to contain the same data as longitude and latitude, so we'll drop the location column
# To drop a column, update the column name in the line below and run this cell
data.drop(["location"], axis = 1, inplace = True)

**We'll clean the data by changing the data type and dropping columns**

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277264 entries, 0 to 277263
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   license_nbr                  277264 non-null  object
 1   license_type                 277264 non-null  object
 2   lic_expir_dd                 256388 non-null  object
 3   license_status               277264 non-null  object
 4   license_creation_date        277264 non-null  object
 5   industry                     277264 non-null  object
 6   business_name                277222 non-null  object
 7   address_city                 276458 non-null  object
 8   address_state                276242 non-null  object
 9   address_zip                  276417 non-null  object
 10  business_name_2              50556 non-null   object
 11  address_building             183070 non-null  object
 12  address_street_name          184137 non-null  object
 13  contact_phone 

In [20]:
# we'll change the longitude and latitude data types to float and the license creation and expiration dates to python date objects

data["longitude"] = data["longitude"].astype(float)
data["latitude"] = data["latitude"].astype(float)

data["lic_expir_dd"] = pd.to_datetime(data["lic_expir_dd"])
data["license_creation_date"] = pd.to_datetime(data["license_creation_date"])

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277264 entries, 0 to 277263
Data columns (total 31 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   license_nbr                  277264 non-null  object        
 1   license_type                 277264 non-null  object        
 2   lic_expir_dd                 256388 non-null  datetime64[ns]
 3   license_status               277264 non-null  object        
 4   license_creation_date        277264 non-null  datetime64[ns]
 5   industry                     277264 non-null  object        
 6   business_name                277222 non-null  object        
 7   address_city                 276458 non-null  object        
 8   address_state                276242 non-null  object        
 9   address_zip                  276417 non-null  object        
 10  business_name_2              50556 non-null   object        
 11  address_building          

In [22]:
# it appears that for records in which business_name_2 is populated, an individual's name is populated in business_name for some of these records

data.iloc[50:100,6:][data["business_name_2"].notnull()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,business_name,address_city,address_state,address_zip,business_name_2,address_building,address_street_name,contact_phone,address_borough,detail,...,census_tract,longitude,latitude,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,detail_2,address_street_name_2
52,Recovery Racing V LLC,NEW YORK,NY,10019,Maserati of Manhattan,619,W 54TH ST,NaN,Manhattan,1,...,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,2474 QUAD VENTURES INC.,BROOKLYN,NY,11234,7 Eleven Store #35049A,2474,FLATBUSH AVE,718-252-0549,Brooklyn,3,...,662,-73.922839,40.610112,13825,5,2,8,38,NaN,NaN
64,IL RIFUGIO INC.,NEW YORK,NY,10024,TARALLUCCI E VINO,475,COLUMBUS AVE,2123625454,Manhattan,1,...,169,-73.973697,40.784381,12421,20,4,19,12,"Sidewalk Cafe Type: Unenclosed, Square Feet: 1...",NaN
75,"PYUN, KISUN",STATEN ISLAND,NY,10302,7-ELEVEN,1440,FOREST AVE,7184201340,Staten Island,5,...,201,-74.136577,40.624588,10370,4,1,13,75,NaN,NaN
81,STARBUCKS CORPORATION,BROOKLYN,NY,11209,STARBUCKS COFFEE COMPANY,7419,3RD AVE,2126131280,Brooklyn,3,...,66,-74.027576,40.631961,17216,10,2,44,41,"Sidewalk Cafe Type: Enclosed, Square Feet: 260...",NaN
84,MORGAN GLOBAL GROUP LLC,NEW YORK,NY,10019,CITY ZEN,322,W 57TH ST,9178306552,Manhattan,1,...,NaN,-73.983755,40.767077,12081,12,4,10,10,NaN,NaN
88,"JUAN, CARLOS FELIZ",NEW YORK,NY,10027,JC CENTER,27A,W 125TH ST,6463186534,Manhattan,1,...,NaN,-73.943176,40.806811,12424,18,4,36,18,NaN,NaN
91,YAEL A ROSARIO AND MELVIN A GRULLART,BRONX,NY,10459,TWIN AUTO SOUND PLUS,1292A,WESTCHESTER AVE,6466259230,Bronx,2,...,NaN,-73.886743,40.827319,10937,8,5,43,24,NaN,NaN
96,WARRANTY LOGISTICS LLC,BROOKLYN,NY,11239,"Warranty Logistics, LLC",494,GATEWAY DR,6154453228,Brooklyn,3,...,NaN,-73.876830,40.652685,17215,45,2,25,47,NaN,NaN


In [23]:
# we'll create a business name column that would take the business_name_2 if the value is not null, otherwise it would take the value in business_name

# create a function that we'll then apply to each of the rows of the data frame for a newly defined 'adjusted business name' column

def business_name_adj(df_row):
  # we initially used notnull to test whether cell value was null; however, the NaN were treated as float and return an error
  # we instead converted the value for business_name_2 as string before testing whether it's a 'null' value
  if str(df_row["business_name_2"]) != 'nan':
    return df_row["business_name_2"]
  else:
    return df_row["business_name"]  

In [24]:
# we'll apply the previously created function to calculate a new adj_business_name column

data["adj_business_name"] = data.apply(business_name_adj, axis=1)

In [25]:
# we then check whether the logic has been applied correctly to calculate the new column

data[["business_name", "business_name_2", "adj_business_name"]]

,business_name,business_name_2,adj_business_name
0,"YILMAZ, SALIH",NaN,"YILMAZ, SALIH"
1,"BARBARINO, JOHN JR.",JOHN BARBARINO JR_HOME IMPROVEMENT,JOHN BARBARINO JR_HOME IMPROVEMENT
2,"HITE CONSTRUCTION, INC.",NaN,"HITE CONSTRUCTION, INC."
3,"Nieciak, Joseph",NaN,"Nieciak, Joseph"
4,"Arcaro, Tyler",NaN,"Arcaro, Tyler"
...,...,...,...
277259,DAUGHTERS & SONS PROPERTIES MANAGEMENT LLC,DIAL A PROFESSIONAL CONTRACTOR,DIAL A PROFESSIONAL CONTRACTOR
277260,"MALDONADO, WILLIAM",NaN,"MALDONADO, WILLIAM"
277261,CHOSEN VARIETY INC,NaN,CHOSEN VARIETY INC
277262,"CORNICK, MICHAEL",NaN,"CORNICK, MICHAEL"


In [26]:
data.columns

Index(['license_nbr', 'license_type', 'lic_expir_dd', 'license_status',
       'license_creation_date', 'industry', 'business_name', 'address_city',
       'address_state', 'address_zip', 'business_name_2', 'address_building',
       'address_street_name', 'contact_phone', 'address_borough', 'detail',
       'community_board', 'council_district', 'bin', 'bbl', 'nta',
       'census_tract', 'longitude', 'latitude', ':@computed_region_efsh_h5xi',
       ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q',
       ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih',
       'detail_2', 'address_street_name_2', 'adj_business_name'],
      dtype='object')

In [27]:
# we'll now drop columns we don't need

drop_columns = ["business_name", "business_name_2", "contact_phone", 
                "detail", "community_board", "council_district", "bin", "bbl", "nta", "census_tract", ':@computed_region_efsh_h5xi', 
                ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih', 
                'detail_2', 'address_street_name_2', 'address_city', 'address_building', 'address_street_name']

for column in drop_columns:
    try:
        data.drop(column, axis = 1, inplace = True)
    except:
        print(f"unable to drop {column}")

print(f"columns left in dataframe: {data.columns}")

columns left in dataframe: Index(['license_nbr', 'license_type', 'lic_expir_dd', 'license_status',
       'license_creation_date', 'industry', 'address_state', 'address_zip',
       'address_borough', 'longitude', 'latitude', 'adj_business_name'],
      dtype='object')


**We'll now create the license dimension**

In [28]:
# first, copy the entire table
license_dim = data.copy()

In [29]:
# second, subset for only the wanted columns in the dimension
license_dim = license_dim[["license_nbr",
                             "license_type",
                             "license_status"]]

In [30]:
# third, drop duplicate rows in dimension
# each unique row will have a unique license number, so we'll only use the license number column in the drop duplicate function
unique_row = ["license_nbr"]
license_dim = license_dim.drop_duplicates(subset = unique_row, keep = 'first')
license_dim = license_dim.reset_index(drop = True)
license_dim

,license_nbr,license_type,license_status
0,2107592-DCA,Individual,Active
1,0967332-DCA,Business,Inactive
2,1057563-DCA,Business,Active
3,2103411-DCA,Individual,Active
4,2060087-DCA,Individual,Active
...,...,...,...
235879,1336282-DCA,Business,Active
235880,2067116-DCA,Individual,Inactive
235881,1053330-DCA,Business,Active
235882,2081684-DCA,Individual,Active


In [31]:
# fourth, add license_id as a surrogate key
license_dim.insert(0, 'license_id', range(1, 1 + len(license_dim)))
license_dim

,license_id,license_nbr,license_type,license_status
0,1,2107592-DCA,Individual,Active
1,2,0967332-DCA,Business,Inactive
2,3,1057563-DCA,Business,Active
3,4,2103411-DCA,Individual,Active
4,5,2060087-DCA,Individual,Active
...,...,...,...,...
235879,235880,1336282-DCA,Business,Active
235880,235881,2067116-DCA,Individual,Inactive
235881,235882,1053330-DCA,Business,Active
235882,235883,2081684-DCA,Individual,Active


In [32]:
#create a copy of the data frame in case we need to revert changes

data_before_fks = data.copy()

In [33]:
# fifth, add the license_id to the data table
data = data.merge(license_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

data.head(100)

,license_nbr,license_type_x,lic_expir_dd,license_status_x,license_creation_date,industry,address_state,address_zip,address_borough,longitude,latitude,adj_business_name,license_id,license_type_y,license_status_y
0,2107592-DCA,Individual,2023-04-30,Active,2022-07-19,Pedicab Driver,NY,11104,NaN,NaN,NaN,"YILMAZ, SALIH",1,Individual,Active
1,0967332-DCA,Business,2017-02-28,Inactive,2010-02-04,Home Improvement Contractor,NJ,07726,Outside NYC,NaN,NaN,JOHN BARBARINO JR_HOME IMPROVEMENT,2,Business,Inactive
2,1057563-DCA,Business,2023-02-28,Active,2000-07-27,Home Improvement Contractor,NJ,07430,Outside NYC,NaN,NaN,"HITE CONSTRUCTION, INC.",3,Business,Active
3,2103411-DCA,Individual,2024-03-31,Active,2022-01-11,Sightseeing Guide,NY,11214,NaN,NaN,NaN,"Nieciak, Joseph",4,Individual,Active
4,2060087-DCA,Individual,2024-03-31,Active,2017-10-31,Sightseeing Guide,NY,10028,NaN,NaN,NaN,"Arcaro, Tyler",5,Individual,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2107553-DCA,Business,2023-02-28,Active,2022-07-18,Home Improvement Contractor,NY,11716,Outside NYC,NaN,NaN,"CLEAResult East Operating, LLC",96,Business,Active
96,2052462-DCA,Business,2024-06-30,Active,2017-05-04,Electronic & Appliance Service,NY,11239,Brooklyn,-73.876830,40.652685,"Warranty Logistics, LLC",97,Business,Active
97,2010802-DCA,Business,2020-05-25,Inactive,2014-07-17,Sidewalk Cafe,NY,11102,Queens,-73.921495,40.764032,"ASTORIA BRICK, LLC",98,Business,Inactive
98,2107533-DCA,Business,2023-02-28,Active,2022-07-15,Home Improvement Contractor,NY,10031,Manhattan,-73.940838,40.827796,FAFF CITY LLC,99,Business,Active


**We'll now create the company dimension**

In [34]:
# first, copy the entire table
company_dim = data.copy()

In [35]:
# second, subset for only the wanted columns in the dimension
company_dim = company_dim[["adj_business_name",
                             "industry"]]

In [36]:
# third, drop duplicate rows in dimension
unique_row_company = ["adj_business_name", "industry"]
company_dim = company_dim.drop_duplicates(subset = unique_row_company, keep = 'first')
company_dim = company_dim.reset_index(drop = True)
company_dim

,adj_business_name,industry
0,"YILMAZ, SALIH",Pedicab Driver
1,JOHN BARBARINO JR_HOME IMPROVEMENT,Home Improvement Contractor
2,"HITE CONSTRUCTION, INC.",Home Improvement Contractor
3,"Nieciak, Joseph",Sightseeing Guide
4,"Arcaro, Tyler",Sightseeing Guide
...,...,...
210181,"NEW JUBILEE NEWS 11, INC",Tobacco Retail Dealer
210182,DIAL A PROFESSIONAL CONTRACTOR,Home Improvement Contractor
210183,CHOSEN VARIETY INC,Tobacco Retail Dealer
210184,"CORNICK, MICHAEL",Tow Truck Driver


In [37]:
# fourth, add company_id as a surrogate key
company_dim.insert(0, 'company_id', range(1, 1 + len(company_dim)))
company_dim

,company_id,adj_business_name,industry
0,1,"YILMAZ, SALIH",Pedicab Driver
1,2,JOHN BARBARINO JR_HOME IMPROVEMENT,Home Improvement Contractor
2,3,"HITE CONSTRUCTION, INC.",Home Improvement Contractor
3,4,"Nieciak, Joseph",Sightseeing Guide
4,5,"Arcaro, Tyler",Sightseeing Guide
...,...,...,...
210181,210182,"NEW JUBILEE NEWS 11, INC",Tobacco Retail Dealer
210182,210183,DIAL A PROFESSIONAL CONTRACTOR,Home Improvement Contractor
210183,210184,CHOSEN VARIETY INC,Tobacco Retail Dealer
210184,210185,"CORNICK, MICHAEL",Tow Truck Driver


In [38]:
# fifth, add the company_id to the data table
data = data.merge(company_dim,
                  left_on = unique_row_company,
                  right_on = unique_row_company,
                  how = 'left')

data.head(100)

,license_nbr,license_type_x,lic_expir_dd,license_status_x,license_creation_date,industry,address_state,address_zip,address_borough,longitude,latitude,adj_business_name,license_id,license_type_y,license_status_y,company_id
0,2107592-DCA,Individual,2023-04-30,Active,2022-07-19,Pedicab Driver,NY,11104,NaN,NaN,NaN,"YILMAZ, SALIH",1,Individual,Active,1
1,0967332-DCA,Business,2017-02-28,Inactive,2010-02-04,Home Improvement Contractor,NJ,07726,Outside NYC,NaN,NaN,JOHN BARBARINO JR_HOME IMPROVEMENT,2,Business,Inactive,2
2,1057563-DCA,Business,2023-02-28,Active,2000-07-27,Home Improvement Contractor,NJ,07430,Outside NYC,NaN,NaN,"HITE CONSTRUCTION, INC.",3,Business,Active,3
3,2103411-DCA,Individual,2024-03-31,Active,2022-01-11,Sightseeing Guide,NY,11214,NaN,NaN,NaN,"Nieciak, Joseph",4,Individual,Active,4
4,2060087-DCA,Individual,2024-03-31,Active,2017-10-31,Sightseeing Guide,NY,10028,NaN,NaN,NaN,"Arcaro, Tyler",5,Individual,Active,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2107553-DCA,Business,2023-02-28,Active,2022-07-18,Home Improvement Contractor,NY,11716,Outside NYC,NaN,NaN,"CLEAResult East Operating, LLC",96,Business,Active,95
96,2052462-DCA,Business,2024-06-30,Active,2017-05-04,Electronic & Appliance Service,NY,11239,Brooklyn,-73.876830,40.652685,"Warranty Logistics, LLC",97,Business,Active,96
97,2010802-DCA,Business,2020-05-25,Inactive,2014-07-17,Sidewalk Cafe,NY,11102,Queens,-73.921495,40.764032,"ASTORIA BRICK, LLC",98,Business,Inactive,97
98,2107533-DCA,Business,2023-02-28,Active,2022-07-15,Home Improvement Contractor,NY,10031,Manhattan,-73.940838,40.827796,FAFF CITY LLC,99,Business,Active,98


**We'll now create the location dimension**

In [39]:
# first, copy the entire table
location_dim = data.copy()

In [40]:
# second, subset for only the wanted columns in the dimension
location_dim = location_dim[["address_zip",
                             "address_borough",
                             "address_state"]]

In [41]:
# third, drop duplicate rows in dimension
unique_row_location = ["address_zip"]
location_dim = location_dim.drop_duplicates(subset = unique_row_location, keep = 'first')
location_dim = location_dim.reset_index(drop = True)
location_dim

,address_zip,address_borough,address_state
0,11104,NaN,NY
1,07726,Outside NYC,NJ
2,07430,Outside NYC,NJ
3,11214,NaN,NY
4,10028,NaN,NY
...,...,...,...
3794,10542,NaN,NY
3795,30274,NaN,GA
3796,18651,NaN,PA
3797,12491,Outside NYC,NY


In [42]:
# fourth, add location_id as a surrogate key
location_dim.insert(0, 'location_id', range(1, 1 + len(location_dim)))
location_dim

,location_id,address_zip,address_borough,address_state
0,1,11104,NaN,NY
1,2,07726,Outside NYC,NJ
2,3,07430,Outside NYC,NJ
3,4,11214,NaN,NY
4,5,10028,NaN,NY
...,...,...,...,...
3794,3795,10542,NaN,NY
3795,3796,30274,NaN,GA
3796,3797,18651,NaN,PA
3797,3798,12491,Outside NYC,NY


In [43]:
data_before_fks = data.copy()

In [44]:
# fifth, add the location_id to the data table
data = data.merge(location_dim,
                  left_on = unique_row_location,
                  right_on = unique_row_location,
                  how = 'left')

data.head(100)

,license_nbr,license_type_x,lic_expir_dd,license_status_x,license_creation_date,industry,address_state_x,address_zip,address_borough_x,longitude,latitude,adj_business_name,license_id,license_type_y,license_status_y,company_id,location_id,address_borough_y,address_state_y
0,2107592-DCA,Individual,2023-04-30,Active,2022-07-19,Pedicab Driver,NY,11104,NaN,NaN,NaN,"YILMAZ, SALIH",1,Individual,Active,1,1,NaN,NY
1,0967332-DCA,Business,2017-02-28,Inactive,2010-02-04,Home Improvement Contractor,NJ,07726,Outside NYC,NaN,NaN,JOHN BARBARINO JR_HOME IMPROVEMENT,2,Business,Inactive,2,2,Outside NYC,NJ
2,1057563-DCA,Business,2023-02-28,Active,2000-07-27,Home Improvement Contractor,NJ,07430,Outside NYC,NaN,NaN,"HITE CONSTRUCTION, INC.",3,Business,Active,3,3,Outside NYC,NJ
3,2103411-DCA,Individual,2024-03-31,Active,2022-01-11,Sightseeing Guide,NY,11214,NaN,NaN,NaN,"Nieciak, Joseph",4,Individual,Active,4,4,NaN,NY
4,2060087-DCA,Individual,2024-03-31,Active,2017-10-31,Sightseeing Guide,NY,10028,NaN,NaN,NaN,"Arcaro, Tyler",5,Individual,Active,5,5,NaN,NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2107553-DCA,Business,2023-02-28,Active,2022-07-18,Home Improvement Contractor,NY,11716,Outside NYC,NaN,NaN,"CLEAResult East Operating, LLC",96,Business,Active,95,77,Outside NYC,NY
96,2052462-DCA,Business,2024-06-30,Active,2017-05-04,Electronic & Appliance Service,NY,11239,Brooklyn,-73.876830,40.652685,"Warranty Logistics, LLC",97,Business,Active,96,78,Brooklyn,NY
97,2010802-DCA,Business,2020-05-25,Inactive,2014-07-17,Sidewalk Cafe,NY,11102,Queens,-73.921495,40.764032,"ASTORIA BRICK, LLC",98,Business,Inactive,97,79,Queens,NY
98,2107533-DCA,Business,2023-02-28,Active,2022-07-15,Home Improvement Contractor,NY,10031,Manhattan,-73.940838,40.827796,FAFF CITY LLC,99,Business,Active,98,58,Manhattan,NY


**We'll now create the date dimension**

In [46]:
# calculate the max and min dates used in license expiration and creation columns so we know the range for the date dimension in the next step

min_date = min(data["lic_expir_dd"].append(data["license_creation_date"]))
print("The earliest date we need to include in the date dimension is: " + str(min_date))

max_date = max(data["lic_expir_dd"].append(data["license_creation_date"]))
print("The latest date we need to include in the date dimension is: " + str(max_date))


The earliest date we need to include in the date dimension is: 1956-10-19 00:00:00
The latest date we need to include in the date dimension is: 2027-02-28 00:00:00


In [47]:
# create a list of dates by running a SQL query in BigQuery

sql_query = """
            SELECT
              CONCAT (FORMAT_DATE("%Y",d),FORMAT_DATE("%m",d),FORMAT_DATE("%d",d)) as date_id,
              d AS full_date,
              FORMAT_DATE('%w', d) AS week_day,
              FORMAT_DATE('%A', d) AS day_name,
              FORMAT_DATE('%B', d) as month_name,
              FORMAT_DATE('%Q', d) as fiscal_qtr,
              FORMAT_DATE('%Y', d) AS year,
            FROM (
              SELECT
                *
              FROM
                UNNEST(GENERATE_DATE_ARRAY('1956-01-01', '2027-12-31', INTERVAL 1 DAY)) AS d )
            """

# store extracted data in new dataframe
date_dim = bigquery_client.query(sql_query).to_dataframe()

# validate that > 0 rows have been extracted and return dataframe
if len(date_dim) > 0:
    print(f"date dimension created successfully, shape: {date_dim.shape}")
else:
    print("date dimension FAILED")

date dimension created successfully, shape: (26298, 7)


In [50]:
date_dim

,date_id,full_date,week_day,day_name,month_name,fiscal_qtr,year
0,19560101,1956-01-01,0,Sunday,January,1,1956
1,19560102,1956-01-02,1,Monday,January,1,1956
2,19560103,1956-01-03,2,Tuesday,January,1,1956
3,19560104,1956-01-04,3,Wednesday,January,1,1956
4,19560105,1956-01-05,4,Thursday,January,1,1956
...,...,...,...,...,...,...,...
26293,20271227,2027-12-27,1,Monday,December,4,2027
26294,20271228,2027-12-28,2,Tuesday,December,4,2027
26295,20271229,2027-12-29,3,Wednesday,December,4,2027
26296,20271230,2027-12-30,4,Thursday,December,4,2027


In [65]:
data_trial = data.copy()

In [115]:
# create date_id column in the fact table for the license expiration and license creation dates
# we'll use the "strftime" method per https://www.geeksforgeeks.org/python-strftime-function/

# upon further data profiling, we discovered that ~21k entries have NaT for license_expir_dd, which is not accepted by strftime
# we will create a function to set NaT to equal 99999999

def create_date_id(row, column_name):
  try:
    return pd.to_datetime(row[column_name]).strftime("%Y%m%d")
  except:
    return "99999999"


In [118]:
# create date ids for each for the license creation and license expiration columns

data["license_exp_date_id"] = data.apply(create_date_id, column_name='lic_expir_dd', axis=1)

data["license_creation_date_id"] = data.apply(create_date_id, column_name='license_creation_date', axis=1)


In [119]:
data.head()

,license_nbr,license_type_x,lic_expir_dd,license_status_x,license_creation_date,industry,address_state_x,address_zip,address_borough_x,longitude,...,adj_business_name,license_id,license_type_y,license_status_y,company_id,location_id,address_borough_y,address_state_y,license_exp_date_id,license_creation_date_id
0,2107592-DCA,Individual,2023-04-30,Active,2022-07-19,Pedicab Driver,NY,11104,NaN,NaN,...,"YILMAZ, SALIH",1,Individual,Active,1,1,NaN,NY,20230430,20220719
1,0967332-DCA,Business,2017-02-28,Inactive,2010-02-04,Home Improvement Contractor,NJ,07726,Outside NYC,NaN,...,JOHN BARBARINO JR_HOME IMPROVEMENT,2,Business,Inactive,2,2,Outside NYC,NJ,20170228,20100204
2,1057563-DCA,Business,2023-02-28,Active,2000-07-27,Home Improvement Contractor,NJ,07430,Outside NYC,NaN,...,"HITE CONSTRUCTION, INC.",3,Business,Active,3,3,Outside NYC,NJ,20230228,20000727
3,2103411-DCA,Individual,2024-03-31,Active,2022-01-11,Sightseeing Guide,NY,11214,NaN,NaN,...,"Nieciak, Joseph",4,Individual,Active,4,4,NaN,NY,20240331,20220111
4,2060087-DCA,Individual,2024-03-31,Active,2017-10-31,Sightseeing Guide,NY,10028,NaN,NaN,...,"Arcaro, Tyler",5,Individual,Active,5,5,NaN,NY,20240331,20171031


**Create the business registration fact table by taking only columns we need**

In [120]:
# take a subset of fact_table for only the needed columns:
# which are keys and measures
business_lic_fact_table = data[["license_id", "license_creation_date_id", "license_exp_date_id", "company_id", "location_id"]]
business_lic_fact_table

,license_id,license_creation_date_id,license_exp_date_id,company_id,location_id
0,1,20220719,20230430,1,1
1,2,20100204,20170228,2,2
2,3,20000727,20230228,3,3
3,4,20220111,20240331,4,4
4,5,20171031,20240331,5,5
...,...,...,...,...,...
277259,235880,20091022,20250228,210183,163
277260,235881,20180301,20221031,209495,10
277261,235882,20001120,20241231,210184,259
277262,235883,20190128,20241031,210185,222


**We'll now create the population fact table. We found the source from**

https://data.census.gov/cedsci/table?q=All%205-digit%20ZIP%20Code%20Tabulation%20Areas%20within%20United%20States&tid=ACSST5Y2020.S0101

In [157]:
# import the population data set that we downloaded from the Census website
# the data is the population recorded in 2020 for each zip code in the US
census_data = pd.read_csv(r'ACSST5Y2020.S0101-Data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,610,611,612,6

In [158]:
census_data.head()

,GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C01_001MA,S0101_C01_001EA,S0101_C01_002E,S0101_C01_002EA,S0101_C01_002M,S0101_C01_002MA,...,S0101_C06_036MA,S0101_C06_037E,S0101_C06_037EA,S0101_C06_037M,S0101_C06_037MA,S0101_C06_038E,S0101_C06_038EA,S0101_C06_038M,S0101_C06_038MA,Unnamed: 914
0,Geography,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total!!Total population,Annotation of Margin of Error!!Total!!Total po...,Annotation of Estimate!!Total!!Total population,Estimate!!Total!!Total population!!AGE!!Under ...,Annotation of Estimate!!Total!!Total populatio...,Margin of Error!!Total!!Total population!!AGE!...,Annotation of Margin of Error!!Total!!Total po...,...,Annotation of Margin of Error!!Percent Female!...,Estimate!!Percent Female!!Total population!!PE...,Annotation of Estimate!!Percent Female!!Total ...,Margin of Error!!Percent Female!!Total populat...,Annotation of Margin of Error!!Percent Female!...,Estimate!!Percent Female!!Total population!!PE...,Annotation of Estimate!!Percent Female!!Total ...,Margin of Error!!Percent Female!!Total populat...,Annotation of Margin of Error!!Percent Female!...,NaN
1,8600000US00601,ZCTA5 00601,16773,259,NaN,NaN,725,NaN,4,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
2,8600000US00602,ZCTA5 00602,37083,149,NaN,NaN,1386,NaN,30,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
3,8600000US00603,ZCTA5 00603,45652,784,NaN,NaN,1871,NaN,102,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
4,8600000US00606,ZCTA5 00606,6231,337,NaN,NaN,244,NaN,34,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN


In [159]:
# we'll only use the zip code and total population and drop the remaining columns
# rename the columns

census_data = census_data[['NAME', 'S0101_C01_001E']]
census_data.rename(columns={'NAME': 'zip_code', 'S0101_C01_001E': 'population'}, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [160]:
# drop the first row

census_data.drop(index=census_data.index[0], axis=0, inplace=True)

In [161]:
# reindex 
census_data.reset_index(inplace=True)

In [162]:
# create a column of the census date
# we will directly create a census date id and populate it 

census_data["census_date_id"] = "20200101"

In [163]:
census_data

,index,zip_code,population,census_date_id
0,1,ZCTA5 00601,16773,20200101
1,2,ZCTA5 00602,37083,20200101
2,3,ZCTA5 00603,45652,20200101
3,4,ZCTA5 00606,6231,20200101
4,5,ZCTA5 00610,26502,20200101
...,...,...,...,...
33115,33116,ZCTA5 99923,12,20200101
33116,33117,ZCTA5 99925,990,20200101
33117,33118,ZCTA5 99926,1582,20200101
33118,33119,ZCTA5 99927,0,20200101


In [164]:
#remove the redundant index column
#take only the five digit for the zip code column

census_data = census_data[["zip_code", "population", "census_date_id"]]

census_data["zip_code"] = census_data["zip_code"].str[-5:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [165]:
# merge the location_id to the census fact table

census_data = census_data.merge(location_dim,
                  left_on = ["zip_code"],
                  right_on = ["address_zip"],
                  how = 'left')

# the census data includes zip codes across the country, including those that are not in the location dimension
census_data.head()

#ensure that population is integers
census_data["population"] = census_data["population"].astype(int)

,zip_code,population,census_date_id,location_id,address_zip,address_borough,address_state
0,00601,16773,20200101,NaN,NaN,NaN,NaN
1,00602,37083,20200101,NaN,NaN,NaN,NaN
2,00603,45652,20200101,NaN,NaN,NaN,NaN
3,00606,6231,20200101,NaN,NaN,NaN,NaN
4,00610,26502,20200101,NaN,NaN,NaN,NaN


In [166]:
census_data[census_data["location_id"].notnull()]

,zip_code,population,census_date_id,location_id,address_zip,address_borough,address_state
9,00623,41472,20200101,2976.0,00623,NaN,PR
13,00637,21957,20200101,3233.0,00637,NaN,PR
94,00911,7157,20200101,2624.0,00911,NaN,PUERTO RICO
122,00968,3603,20200101,3782.0,00968,Outside NYC,PR
132,01002,30099,20200101,3283.0,01002,NaN,MA
...,...,...,...,...,...,...,...
32626,98632,52489,20200101,2845.0,98632,Outside NYC,WA
32655,98684,31805,20200101,2587.0,98684,Outside NYC,WA
32656,98685,29788,20200101,2555.0,98685,Outside NYC,WA
32831,99202,21488,20200101,3605.0,99202,NaN,WA


**Create the population fact table by including only columns we need**

In [167]:
population_fact_table = census_data[["census_date_id", "location_id", "population"]]
population_fact_table

,census_date_id,location_id,population
0,20200101,NaN,16773
1,20200101,NaN,37083
2,20200101,NaN,45652
3,20200101,NaN,6231
4,20200101,NaN,26502
...,...,...,...
33115,20200101,NaN,12
33116,20200101,NaN,990
33117,20200101,NaN,1582
33118,20200101,NaN,0


**Upload the dimension and fact tables onto BigQuery**

In [171]:
# create a function to load dataframes to BigQuery

def load_table_to_bigquery(df,
                          table_name,
                          dataset_id):

    dataset_id = dataset_id 

    dataset_ref = bigquery_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"

    upload_table_name = f"{dataset_id}.{table_name}"
    
    load_job = bigquery_client.load_table_from_dataframe(df,
                                                upload_table_name,
                                                job_config = job_config)
        
    print(f"completed job {load_job}")

In [172]:
dataset_id = r"cis9440-361702.nyc_business_registrations"

In [173]:
load_table_to_bigquery(df = license_dim,
                       table_name = "license_dim",
                       dataset_id = dataset_id)

completed job LoadJob<project=cis9440-361702, location=US, id=ed029234-14d1-4ab9-ab92-351f5be69042>


In [174]:
load_table_to_bigquery(df = company_dim,
                       table_name = "company_dim",
                       dataset_id = dataset_id)

completed job LoadJob<project=cis9440-361702, location=US, id=77df745b-1248-4c51-a457-162e310cc8a1>


In [175]:
load_table_to_bigquery(df = location_dim,
                       table_name = "location_dim",
                       dataset_id = dataset_id)

completed job LoadJob<project=cis9440-361702, location=US, id=78346969-adff-4e74-81e1-1b133f2f07e8>


In [176]:
load_table_to_bigquery(df = business_lic_fact_table,
                       table_name = "business_license_facts",
                       dataset_id = dataset_id)

completed job LoadJob<project=cis9440-361702, location=US, id=b35b9501-b832-48f6-84da-983c10573960>


In [178]:
load_table_to_bigquery(df = date_dim,
                       table_name = "date_dim",
                       dataset_id = dataset_id)

completed job LoadJob<project=cis9440-361702, location=US, id=58487afa-dbe0-4d74-8fcf-69445564d979>


In [187]:
load_table_to_bigquery(df = population_fact_table,
                       table_name = "population_facts",
                       dataset_id = dataset_id)

completed job LoadJob<project=cis9440-361702, location=US, id=7f270753-d01b-408d-bd9a-ca4d7b8ab44a>
